# Exploratory Analysis of NBA API available data
## Goal: Create shot charts and investigate other available data to perform spatial analysis

In [3]:
import pandas as pd 
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats
import plotly.express as px
import plotly.graph_objects as go

In [4]:
# From: https://github.com/hkair/nba-shotcharts
def get_player_shotchartdetail(player_name, season_id):
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

#### Get shot chart info for Kawhi Leanords 2018-2019 season

In [5]:
player_shotchart_df, league_avg = get_player_shotchartdetail('Kawhi Leonard', '2018-19')
player_shotchart_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021800008,9,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,11,24,...,Center(C),16-24 ft.,21,13,214,1,0,20181017,TOR,CLE
1,Shot Chart Detail,0021800008,28,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,9,47,...,Right Side(R),8-16 ft.,12,118,44,1,0,20181017,TOR,CLE
2,Shot Chart Detail,0021800008,45,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,8,31,...,Center(C),8-16 ft.,12,-34,120,1,0,20181017,TOR,CLE
3,Shot Chart Detail,0021800008,53,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,8,3,...,Center(C),Less Than 8 ft.,1,2,12,1,1,20181017,TOR,CLE
4,Shot Chart Detail,0021800008,98,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,4,44,...,Center(C),Less Than 8 ft.,4,0,41,1,1,20181017,TOR,CLE


#### Check for missing values in XY coordinates 

In [6]:
player_shotchart_df['LOC_X'].isnull().any() or player_shotchart_df['LOC_Y'].isnull().any()

False

In [7]:
player_shotchart_df.columns

Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM'],
      dtype='object')

#### Function also returned league averages for the given season based on zones

In [8]:
league_avg

,GRID_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,FGA,FGM,FG_PCT
0,League Averages,Above the Break 3,Back Court(BC),Back Court Shot,43,2,0.047
1,League Averages,Above the Break 3,Center(C),24+ ft.,15944,5449,0.342
2,League Averages,Above the Break 3,Left Side Center(LC),24+ ft.,22539,7944,0.352
3,League Averages,Above the Break 3,Right Side Center(RC),24+ ft.,21759,7666,0.352
4,League Averages,Backcourt,Back Court(BC),Back Court Shot,466,14,0.030
5,League Averages,In The Paint (Non-RA),Center(C),8-16 ft.,11502,4866,0.423
6,League Averages,In The Paint (Non-RA),Center(C),Less Than 8 ft.,19554,7665,0.392
7,League Averages,In The Paint (Non-RA),Left Side(L),8-16 ft.,1999,797,0.399
8,League Averages,In The Paint (Non-RA),Right Side(R),8-16 ft.,2067,849,0.411
9,League Averages,Left Corner 3,Left Side(L),24+ ft.,9112,3519,0.386


## Shot Chart Creation

#### Function to draw court lines

In [9]:
#From: https://gitlab.com/jphwang/online_articles/-/blob/master/basketball_plots/
def draw_plotly_court(fig, fig_width=600, margins=10):

    import numpy as np

    # From: https://community.plot.ly/t/arc-shape-with-path/7205/5
    def ellipse_arc(x_center=0.0, y_center=0.0, a=10.5, b=10.5, start_angle=0.0, end_angle=2 * np.pi, N=200, closed=False):
        t = np.linspace(start_angle, end_angle, N)
        x = x_center + a * np.cos(t)
        y = y_center + b * np.sin(t)
        path = f'M {x[0]}, {y[0]}'
        for k in range(1, len(t)):
            path += f'L{x[k]}, {y[k]}'
        if closed:
            path += ' Z'
        return path

    fig_height = fig_width * (470 + 2 * margins) / (500 + 2 * margins)
    fig.update_layout(width=fig_width, height=fig_height)

    # Set axes ranges
    fig.update_xaxes(range=[-250 - margins, 250 + margins])
    fig.update_yaxes(range=[-52.5 - margins, 417.5 + margins])

    threept_break_y = 89.47765084
    three_line_col = "#777777"
    main_line_col = "#777777"

    fig.update_layout(
        # Line Horizontal
        margin=dict(l=20, r=20, t=20, b=20),
        paper_bgcolor="white",
        plot_bgcolor="white",
        yaxis=dict(
            scaleanchor="x",
            scaleratio=1,
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        xaxis=dict(
            showgrid=False,
            zeroline=False,
            showline=False,
            ticks='',
            showticklabels=False,
            fixedrange=True,
        ),
        shapes=[
            dict(
                type="rect", x0=-250, y0=-52.5, x1=250, y1=417.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-80, y0=-52.5, x1=80, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="rect", x0=-60, y0=-52.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                # fillcolor='#333333',
                layer='below'
            ),
            dict(
                type="circle", x0=-60, y0=77.5, x1=60, y1=197.5, xref="x", yref="y",
                line=dict(color=main_line_col, width=1),
                # fillcolor='#dddddd',
                layer='below'
            ),
            dict(
                type="line", x0=-60, y0=137.5, x1=60, y1=137.5,
                line=dict(color=main_line_col, width=1),
                layer='below'
            ),

            dict(
                type="rect", x0=-2, y0=-7.25, x1=2, y1=-12.5,
                line=dict(color="#ec7607", width=1),
                fillcolor='#ec7607',
            ),
            dict(
                type="circle", x0=-7.5, y0=-7.5, x1=7.5, y1=7.5, xref="x", yref="y",
                line=dict(color="#ec7607", width=1),
            ),
            dict(
                type="line", x0=-30, y0=-12.5, x1=30, y1=-12.5,
                line=dict(color="#ec7607", width=1),
            ),

            dict(type="path",
                 path=ellipse_arc(a=40, b=40, start_angle=0, end_angle=np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(type="path",
                 path=ellipse_arc(a=237.5, b=237.5, start_angle=0.386283101, end_angle=np.pi - 0.386283101),
                 line=dict(color=main_line_col, width=1), layer='below'),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-220, y0=-52.5, x1=-220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=220, y0=-52.5, x1=220, y1=threept_break_y,
                line=dict(color=three_line_col, width=1), layer='below'
            ),

            dict(
                type="line", x0=-250, y0=227.5, x1=-220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=250, y0=227.5, x1=220, y1=227.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=17.5, x1=-80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=27.5, x1=-80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=57.5, x1=-80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=-90, y0=87.5, x1=-80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=17.5, x1=80, y1=17.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=27.5, x1=80, y1=27.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=57.5, x1=80, y1=57.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),
            dict(
                type="line", x0=90, y0=87.5, x1=80, y1=87.5,
                line=dict(color=main_line_col, width=1), layer='below'
            ),

            dict(type="path",
                 path=ellipse_arc(y_center=417.5, a=60, b=60, start_angle=-0, end_angle=-np.pi),
                 line=dict(color=main_line_col, width=1), layer='below'),

        ]
    )
    return True

In [10]:
fig = go.Figure()
draw_plotly_court(fig)
fig.add_trace(go.Scatter(
    x=player_shotchart_df[player_shotchart_df['SHOT_MADE_FLAG']==1]['LOC_X'],
    y=player_shotchart_df[player_shotchart_df['SHOT_MADE_FLAG']==1]['LOC_Y'],
    mode='markers', name='Make',
    marker=dict(size=5,color='red',
    line=dict(width=1, color='#333333'), symbol='circle'
    )
))
fig.add_trace(go.Scatter(
    x=player_shotchart_df[player_shotchart_df['SHOT_MADE_FLAG']==0]['LOC_X'],
    y=player_shotchart_df[player_shotchart_df['SHOT_MADE_FLAG']==0]['LOC_Y'],
    mode='markers', name='Miss',
    marker=dict(size=5,color='blue',
    line=dict(width=1, color='#333333'), symbol='x'
    )
))
fig.update_layout(
    title={
        'text': "Kawhi Leonard 2018-19 Shot Chart",
        'y':0.98,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
fig.show(config=dict(displayModeBar=False))